In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
train['target'].unique()
train['target']=train['target'].str[-1]
train.pop('id')

y = train['target']
train.pop('target')
X=train
# idx=X[X.duplicated()==True].index
# X.describe()


In [ ]:
# X.drop(X.index[idx], inplace=True)

In [ ]:
# y.drop(y.index[idx], inplace=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, Reshape, BatchNormalization, Add,Dropout, Conv1D,AveragePooling1D,Concatenate, Average, Embedding, GlobalAveragePooling1D, Flatten, AlphaDropout
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
from keras.backend import int_shape

In [ ]:
#X = X.astype('category')
#X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.05,random_state=151,stratify=y)

In [ ]:
# emb = X_train.max(axis=0)
# emb = emb.cumsum(axis=0)
# emb = emb.shift(1)
# emb = emb.fillna(0)
# emb = np.array(emb)
#np.array(X_train).max()
#X[:10]

In [ ]:
# #scaler = RobustScaler()
# #X_train=scaler.fit_transform(X_train)

# y_train=np.array(y_train)
# X_train=np.array(X_train)+1
# X = np.array(X)+1
# #X_train=np.clip(X_train,1,100)
# #X_test = scaler.transform(X_test)

# y_test=np.array(y_test)
# X_test=np.array(X_test)+1
# #X_test=np.clip(X_test,1,100)
# X_train=np.float32(X_train)
# X=np.float32(X)
# y=np.float32(y)
# y_train=np.float32(y_train)
# X_test=np.float32(X_test)
# y_test=np.float32(y_test)
# y_train=y_train-1
# y_test=y_test-1
# y=y-1

In [ ]:
#X_train.min()

In [ ]:
#smote = SMOTE(random_state=12)
#X_train, y_train = smote.fit_resample(X_train, y_train)
#oversample = RandomOverSampler(sampling_strategy='minority')
#X_train, y_train = oversample.fit_resample(X_train, y_train)
#emb_sizes = X_train.max(axis=0)

In [ ]:
#np.unique(y_train, return_counts=True)

In [ ]:
def ClassModel(input_shape):

    
    X_input = Input(input_shape)
    X = Embedding (input_dim=354, output_dim=8)(X_input)

    X = Conv1D(64,1,padding='same',activation='relu')(X)
    X = Conv1D(128,1,padding='same',activation='relu')(X)
    #X = GlobalAveragePooling1D()(X)
    X = Flatten()(X)
    #X = Dropout(0.2)(X)
    X2 = Dense(75, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_a')(X)
    #X2 = BatchNormalization()(X2)
    #X2 = Dropout(0.2)(X2)
    X2 = Activation('relu')(X2)
    X3 = Dense(150, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_b')(X)
    #X3 = BatchNormalization()(X3)
    #X3 = Dropout(0.2)(X3)
    X3 = Activation('relu')(X3)
    X4 = Dense(50, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_c')(X)
    #X4 = BatchNormalization()(X4)
    #X4 = Dropout(0.2)(X4)
    X4 = Activation('relu')(X4)
    X = Concatenate()([X2,X3,X4])
    X = Dense(100, name='lay1',kernel_initializer='glorot_uniform')(X)
    #X = BatchNormalization(axis = -1, name = 'bn1')(X)
    #X = Dropout(0.2)(X)
    X = Activation('relu')(X)
#     X = Dense(300, name='lay2',kernel_initializer='glorot_uniform')(X)
#     X = BatchNormalization(axis = -1, name = 'bn2')(X)
#     X = Dropout(0.4)(X)
#     X = Activation('relu')(X)
#     X = Dense(100, name='lay3',kernel_initializer='glorot_uniform')(X)
#     X = BatchNormalization(axis = -1, name = 'bn3')(X)
#     X = Dropout(0.4)(X)
#     X = Activation('relu')(X)
    X = Dense(50, name='lay4',kernel_initializer='glorot_uniform')(X)
#     X = BatchNormalization(axis = -1, name = 'bn4')(X)
#     X = Dropout(0.4)(X)
    X = Activation('relu')(X)

    X = Dense(9, activation='softmax', name='output')(X)


    model = Model(inputs = X_input, outputs = X, name='ClassModel')

    
    return model

def ResModel(input_shape):

    
    X_input = Input(input_shape)
    X = Embedding (input_dim=354, output_dim=2)(X_input)
    X = Conv1D(16,1,activation='relu')(X)
    X = Flatten()(X)
    X2 = Dense(75, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_a')(X)
    #X2 = BatchNormalization()(X2)
    #X2 = Dropout(0.2)(X2)
    X2 = Activation("relu")(X2)
    X3 = Dense(150, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_b')(X)
    #X3 = BatchNormalization()(X3)
    #X3 = Dropout(0.2)(X3)
    X3 = Activation("relu")(X3)
    X4 = Dense(50, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1_c')(X)
    #X4 = BatchNormalization()(X4)
    #X4 = Dropout(0.2)(X4)
    X4 = Activation("relu")(X4)
    X = Concatenate()([X2,X3,X4])
#     X = Dropout(0.3)(X)
    X = Dense(120, kernel_initializer=tf.keras.initializers.GlorotNormal(),name='lay1')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X1 = Activation("relu")(X)
    X = Dense(120, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay2')(X1)
    
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Dense(120, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay3')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Add()([X1,X])
    X = Dense(60, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay1a')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)

    X2 = Activation("relu")(X)
    X = Dense(60, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay2a')(X1)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Dense(60, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay3a')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Concatenate()([X2,X])
    X = Add()([X,X1])
    X = Dense(30, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay1b')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X3 = Activation("relu")(X)
    X = Dense(30, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay2b')(X1)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Dense(30, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='lay3b')(X)
#     X = BatchNormalization()(X)
#     X = Dropout(0.3)(X)
    X = Activation("relu")(X)
    X = Concatenate()([X3,X2,X])
    X = Add()([X,X1])
    X = Dense(20, kernel_initializer=tf.keras.initializers.GlorotNormal(), name='final')(X)
    
    X = Dense(9, activation='softmax', name='output')(X)


    model = Model(inputs = X_input, outputs = X, name='ResModel')

    
    return model
def EmbeddingBlock(X, emb_size):
    X=Embedding(input_dim=emb_size+1, output_dim=15, embeddings_regularizer='l2')(X)
    X=Flatten()(X)
    X=Dense(1000, kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    X=BatchNormalization(axis=-1)(X)
    X=Dropout(0.4)(X)
    X=Activation('relu')(X)
    return X
    
def MultiEmbModel():
    inputs = []
    embeddings = []
    for i in range(75):
        inputs.append(Input(1,))
    for i,emb_size in enumerate(emb_sizes):
        embeddings.append(EmbeddingBlock(inputs[i],int(emb_size)))
    X = Concatenate()(embeddings)
    #X = Dense(800, name='lay1',kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    #X = BatchNormalization(axis = -1, name = 'bn1')(X)
    #X = Dropout(0.5)(X)
    #X = Activation('relu')(X)
    X = Dense(400, name='lay2',kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    X = BatchNormalization(axis = -1, name = 'bn2')(X)
    X = Dropout(0.5)(X)
    X = Activation('relu')(X)
    X = Dense(200, name='lay3',kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    X = BatchNormalization(axis = -1, name = 'bn3')(X)
    X = Dropout(0.5)(X)
    X = Activation('relu')(X)
    #X = Dense(100, name='lay4',kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    #X = BatchNormalization(axis = -1, name = 'bn4')(X)
    #X = Dropout(0.5)(X)
    #X = Activation('relu')(X)

    X = Dense(9, activation='softmax', name='output')(X)


    model = Model(inputs = inputs, outputs = X, name='EmbModel')

    
    return model

In [ ]:
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.4,
#                             patience=3, min_lr=0.00001, verbose=1)
checkpoint_filepath = '/kaggle/working/ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:

def prediction (X_train, y_train, X_test):
    
    keras.backend.clear_session()

    kfold = StratifiedKFold(n_splits = 10, shuffle=True)

    y_pred = np.zeros((100000,9))
    train_oof = np.zeros((200000,9))

    for idx in kfold.split(X=X_train, y=y_train):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train.iloc[train_idx]
        ytrain = y_train.iloc[train_idx]
        xval = X_train.iloc[val_idx]
        yval = y_train.iloc[val_idx]
        ytrain = ytrain.astype('int')
        yval = yval.astype('int')
        ytrain-=1
        yval-=1
        ytrain = to_categorical(ytrain)
        yval = to_categorical(yval)


        
        checkpoint_filepath = '/kaggle/working/ckpt1'
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
        keras.backend.clear_session()
        # fit model for current fold
        classmodel = ResModel(xtrain.shape[1:])
        classmodel.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.00015), metrics=['accuracy'])
        
        classmodel.fit(xtrain, ytrain,
        batch_size = 128, epochs = 14,
        validation_data=(xval, yval),
        callbacks=[model_checkpoint_callback])
        keras.backend.clear_session()
        classmodel.load_weights(checkpoint_filepath)
        #create predictions
        y_pred += classmodel.predict(X_test)/kfold.n_splits
        #print(y_pred)
               
        val_pred = classmodel.predict(xval)
        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred
        
        # calculate and append logloss
        fold_logloss = metrics.log_loss(yval,val_pred)
        #print(classmodel.evaluate(xval,yval))
        #classmodel.evaluate(xval,yval)
        print("Logloss: {0:0.5f}". format(fold_logloss))
    
    return y_pred, train_oof, classmodel

In [ ]:
X_test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
X_test=X_test.iloc[:,1:]
main_pred, train_oof, classmodel = prediction(X, y, X_test)
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
sub.iloc[:,1:]=main_pred
sub=sub.set_index('id')
sub.to_csv('embmodel.csv')
tf.keras.utils.plot_model(classmodel,show_shapes=True) 
# classModel = ClassModel(X.shape[1:])
# classModel.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))
# classModel.summary()

# resModel = ResModel(X.shape[1:])
# resModel.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.Adadelta(learning_rate=1))





In [ ]:
#tf.keras.utils.plot_model(classmodel,show_shapes=True)

In [ ]:
# history=classModel.fit(X_train,y_train, validation_data=(X_test,y_test),epochs = 15, batch_size=64, shuffle=True, callbacks=[model_checkpoint_callback])

In [ ]:
# classModel.load_weights('/kaggle/working/cp-0013.ckpt')

In [ ]:
# classModel.evaluate(X_test,y_test)

In [ ]:
# X_test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
# X=test.iloc[:,1:]
# #X=scaler.transform(X)
# X=X+1
# #X=np.array(X)
# #X=np.clip(X,1,100)
# #X_pred = []

# #for x in range(75):
#     #X_pred.append(X[:,x])
# y=classModel.predict(X)
# sub = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
# sub.iloc[:,1:]=y
# sub=sub.set_index('id')
# sub.to_csv('embmodel.csv')